In [1]:
import syft as sy

assigning id:<UID:a011b940-1824-4a1a-82e9-a40eff61f836> type:<class 'torch.Tensor'>


### Test 1: Send a Message from a VM Client to a VM

In [2]:
bob_vm = sy.VirtualMachine(name="Bob")
bob_vm_client = bob_vm.get_client()

In [3]:
assert bob_vm.device is None

In [4]:
bob_vm_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

the message is for me!!!
VirtualMachine:Bob:<UID:06f21705-8745-47c9-a8cc-224887566096>


### Test 2: Send a message from a Device Client to a Device

In [5]:
bob_phone = sy.Device(name="Bob's iPhone")
bob_phone_client = bob_phone.get_client()

In [6]:
msg = sy.ReprMessage(address=bob_phone_client)

In [7]:
def print_path(path):
    output = ""
    indent = 0
    for item in path:
        tabs = "".join(["\t"] * indent)
        output += f"{tabs}{item}\n"
        indent += 1
    print(output)

In [8]:
# bobs client has a SoloRoute with a VirtualClientConnection to VirtualServerConnection to bobs_phone
path = [bob_phone_client, bob_phone_client.routes[0], bob_phone_client.routes[0].connection, bob_phone_client.routes[0].connection.server, bob_phone_client.routes[0].connection.server.node]
print_path(path)

bob_phone_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_phone_client))

<DeviceClient:<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>>
	<SoloRoute:d44df485-dd57-425f-81ed-845e2dcb8526>
				Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>

the message is for me!!!
Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>


### Test 2.1: Register a VM on a Device

In [9]:
print(bob_phone_client.routes[0].connection.server.node)
print(bob_phone_client.routes[0].connection.server.node.device)
print(bob_phone_client.device)
print(bob_vm_client)
print(bob_vm_client.vm)
print(bob_vm_client.device) # no device on the vm client yet

Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<VirtualMachineClient id:Bob>
<SpecificLocation:06f21705-8745-47c9-a8cc-224887566096>
None


In [10]:
bob_phone_client.register(client=bob_vm_client)

the message is for me!!!
the message is for me!!!
Updating to Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce> on node VirtualMachine:Bob:<UID:06f21705-8745-47c9-a8cc-224887566096>
child nodes of:Bob


In [11]:
print(bob_phone.store) # bob_vm_client.id is now stored on the device
assert bob_phone.store[bob_vm_client.id].data == bob_vm_client

{<UID:06f21705-8745-47c9-a8cc-224887566096>: <Storable:<VirtualMachineClient id:Bob>>}


### Test 2.2 Test that registration updated "address" variable of VM

In [12]:
assert bob_vm.device is not None
assert bob_vm.device.id == bob_phone_client.device.id
print(bob_vm.device)

<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>


In [13]:
assert bob_vm_client.device is not None
assert bob_vm_client.device.id == bob_phone_client.device.id
print(bob_vm_client.device)

<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>


### Test 3: Send a message from a Device Client to a VM

In [14]:
print(bob_phone_client.routes[0].connection.server.node)
print(bob_phone_client.routes[0].connection.server.node.device)
print(bob_phone_client.device)
print(bob_vm_client)
print(bob_vm_client.vm)
print(bob_vm_client.device)

Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<VirtualMachineClient id:Bob>
<SpecificLocation:06f21705-8745-47c9-a8cc-224887566096>
<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>


In [15]:
print(bob_phone_client.id) # phone client id is phone device
print(bob_vm_client.id) # vm client id is vm

<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<UID:06f21705-8745-47c9-a8cc-224887566096>


In [16]:
# bobs client has a SoloRoute with a VirtualClientConnection to VirtualServerConnection to bobs_phone
path = [
    bob_phone_client,
    bob_phone_client.routes[0],
    bob_phone_client.routes[0].connection,
    bob_phone_client.routes[0].connection.server,
    bob_phone_client.routes[0].connection.server.node,
    bob_phone_client.routes[0].connection.server.node.message_without_reply_forwarding_service,
    bob_phone_client.routes[0].connection.server.node.store[bob_vm_client.id].data
]
print_path(path)

bob_phone_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

<DeviceClient:<SpecificLocation:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>>
	<SoloRoute:d44df485-dd57-425f-81ed-845e2dcb8526>
				Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
						<VirtualMachineClient id:Bob>

the message is not for me... <UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<UID:06f21705-8745-47c9-a8cc-224887566096>
the message is for me!!!
VirtualMachine:Bob:<UID:06f21705-8745-47c9-a8cc-224887566096>


### Test 4: Send a message from a Domain Client to a VM

In [17]:
import syft as sy

In [18]:
bob_domain = sy.Domain(name="Bob's Domain")
bob_domain_client = bob_domain.get_client()

In [19]:
bob_domain_client.register(client=bob_phone_client)

the message is for me!!!
the message is for me!!!
Updating to Domain:Bob's Domain:<UID:a9e7dedd-75be-4b2c-a76a-0e69715496c3> on node Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
child nodes of:Bob's iPhone
	Bob
the message is for me!!!
Updating to Domain:Bob's Domain:<UID:a9e7dedd-75be-4b2c-a76a-0e69715496c3> on node VirtualMachine:Bob:<UID:06f21705-8745-47c9-a8cc-224887566096>
child nodes of:Bob


In [20]:
bob_domain_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm))

the message is not for me... <UID:a9e7dedd-75be-4b2c-a76a-0e69715496c3>
<UID:06f21705-8745-47c9-a8cc-224887566096>
the message is not for me... <UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<UID:06f21705-8745-47c9-a8cc-224887566096>
the message is for me!!!
VirtualMachine:Bob:<UID:06f21705-8745-47c9-a8cc-224887566096>


### Test 5: Send a message from Network Client to a VM

In [21]:
bob_network = sy.Network(name="Bob's Network")
bob_network_client = bob_network.get_client()

In [22]:
bob_network_client.register(client=bob_domain_client)

the message is for me!!!
the message is for me!!!
Updating to Network:Bob's Network:<UID:fb3d94d8-ef62-45f8-b869-734b4d0d2dff> on node Domain:Bob's Domain:<UID:a9e7dedd-75be-4b2c-a76a-0e69715496c3>
child nodes of:Bob's Domain
	Bob's iPhone
the message is for me!!!
Updating to Network:Bob's Network:<UID:fb3d94d8-ef62-45f8-b869-734b4d0d2dff> on node Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
child nodes of:Bob's iPhone
	Bob
the message is for me!!!
Updating to Network:Bob's Network:<UID:fb3d94d8-ef62-45f8-b869-734b4d0d2dff> on node VirtualMachine:Bob:<UID:06f21705-8745-47c9-a8cc-224887566096>
child nodes of:Bob


In [23]:
bob_network_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm))

the message is not for me... <UID:fb3d94d8-ef62-45f8-b869-734b4d0d2dff>
<UID:06f21705-8745-47c9-a8cc-224887566096>
the message is not for me... <UID:a9e7dedd-75be-4b2c-a76a-0e69715496c3>
<UID:06f21705-8745-47c9-a8cc-224887566096>
the message is not for me... <UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>
<UID:06f21705-8745-47c9-a8cc-224887566096>
the message is for me!!!
VirtualMachine:Bob:<UID:06f21705-8745-47c9-a8cc-224887566096>


### Test 5: Use VM Client wtih Torch Tensor

In [24]:
print(type(bob_network_client))
print(bob_network_client.routes)
print(bob_network_client.routes[0])
print(bob_network_client.routes[0].connection)

<class 'syft.core.node.network.client.NetworkClient'>
[<SoloRoute:5c44ff2b-7f0e-46f0-b24a-07e93b600012>]
<SoloRoute:5c44ff2b-7f0e-46f0-b24a-07e93b600012>


### Authorized Messages

In [25]:
from syft.core.node.common.service.authorized_service import HelloRootRequest
from nacl.signing import SigningKey, VerifyKey

In [26]:
bobs_signing_key = SigningKey.generate()
bobs_verification_key = bobs_signing_key.verify_key
print(bobs_signing_key) # private dont share with anyone
print(bobs_verification_key) # public give to anyone who needs to verify your messages

b'\xa6\x97\xc9\xcau^\xffy\xcdE<\xe4\x04\x10)\r\x01\xf4\xcb\xe8?\xa3\xf9~\x93`ly\xb3t\x87\xba'
b'\x07\x98\xf5\xa2\xe5U\xe3\xa8\x1c\x9f\x9c\xfd\xe3^y\x81\x19\xbe\xaa\xadK\r\xd4\xf1\xafL\xaf\x1dy\xbdcu'


In [27]:
secure_msg = HelloRootRequest(username="bob", reply_to=bob_phone_client, address=bob_phone)

In [28]:
sm = secure_msg.serialize(to_binary=True)
print(sm)
print(type(sm))

b'{"objType": "syft.core.node.common.service.authorized_service.HelloRootRequest", "content": "{\\"username\\": \\"bob\\", \\"replyTo\\": {\\"hasDomain\\": true, \\"domain\\": {\\"id\\": {\\"value\\": \\"qefe3XW+Syynag5pcVSWww==\\"}}, \\"hasDevice\\": true, \\"device\\": {\\"id\\": {\\"value\\": \\"LwUtFkolR2WdSGs9Ky44zg==\\"}}}, \\"address\\": {\\"hasNetwork\\": true, \\"network\\": {\\"id\\": {\\"value\\": \\"+z2U2O9iRfi4aXNLTQ0t/w==\\"}}, \\"hasDomain\\": true, \\"domain\\": {\\"id\\": {\\"value\\": \\"qefe3XW+Syynag5pcVSWww==\\"}}, \\"hasDevice\\": true, \\"device\\": {\\"id\\": {\\"value\\": \\"LwUtFkolR2WdSGs9Ky44zg==\\"}}}, \\"msgId\\": {\\"value\\": \\"UVF2XOZlQvywwKEI2BQK5g==\\"}}"}'
<class 'bytes'>


In [29]:
dm = sy.deserialize(blob=sm, from_binary=True)
print(dm)
print(type(dm))

<HelloRootRequest:5151765c-e665-42fc-b0c0-a108d8140ae6>
<class 'syft.core.node.common.service.authorized_service.HelloRootRequest'>


In [30]:
signed_msg = secure_msg.sign_message(signing_key=bobs_signing_key)
signed_msg.address = secure_msg.address
print(signed_msg.address)

Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce>


In [31]:
sig_hello = bob_phone_client.send_signed_msg_with_reply(msg=signed_msg)
print(sig_hello)

hello = sig_hello.inner_message()
print(hello.message)

Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce> <HelloRootRequest:5151765c-e665-42fc-b0c0-a108d8140ae6>
Hello bob


In [32]:
# make a tampered message
tampered_msg = secure_msg.sign_message(signing_key=bobs_signing_key)
tampered_msg.address = secure_msg.address
tampered_msg.signature += b"a"

In [33]:
sig_hello = bob_phone_client.send_signed_msg_with_reply(msg=tampered_msg)
print(sig_hello)

hello = sig_hello.inner_message(allow_invalid=True)
print(hello.message)

Device:Bob's iPhone:<UID:2f052d16-4a25-4765-9d48-6b3d2b2e38ce> <HelloRootRequest:5151765c-e665-42fc-b0c0-a108d8140ae6>
Unauthorized. Go away bob!


In [ ]:
import syft as sy
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()

In [ ]:
import torch as th

In [ ]:
x = th.tensor([1,2,3,4])

blob = x.serialize()

x2 = sy.deserialize(blob=blob)

assert (x == x2).all()

In [ ]:
x.id

In [ ]:
xp = x.send(alice_client)

In [ ]:
xp.id_at_location

In [ ]:
y = xp + xp

In [ ]:
y.get()

In [ ]:
alice.store

In [ ]:
del xp

In [ ]:
alice.store

In [ ]:
ptr = alice_client.torch.zeros(10,10)

In [ ]:
import numpy as np
import syft as sy

In [ ]:
x = np.array([1,2,3,4]).send(alice_client)

In [ ]:
x.get()

In [ ]:
x = np.array([1,2,3,4])

In [ ]:
x.serialize()

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

    def __repr__(self):
        return f'User {self.name}'

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
user = User(name='John Snow', password='johnspassword')
session.add(user)

print(user.id)  # None

In [ ]:
session.commit()

In [ ]:
query = session.query(User).filter_by(name='John Snow')

In [ ]:
query.count()